In [3]:
#Importing libraries 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/cleaning/all_data.csv')